In [ ]:
import numpy as np
import pyci
import psi4

In [ ]:
# ! s-type d-aug-cc-pVQZ functions
# ! s-type ARO functions
# ! p-type d-aug-cc-pVQZ functions
# ! p-type ARO functions
# ! d-type d-aug-cc-pVQZ functions
# ! d-type ARO functions
# ! f-type d-aug-cc-pVQZ functions
# ! f-type ARO functions
basis_dict = {
      'basstring': """
****
He     0
S    7   1.00
    528.5000000              0.0009400
     79.3100000              0.0072140
     18.0500000              0.0359750
      5.0850000              0.1277820
      1.6090000              0.3084700
      0.5363000              0.4530520
      0.1833000              0.2388840
S    1   1.00
      1.6090000              1.0000000
S    1   1.00
      0.5363000              1.0000000
S    1   1.00
      0.1833000              1.0000000
S    1   1.00
      0.0481900              1.0000000
S    1   1.00
      0.0127                 1.0000000
S    1   1.00
      4.142630e-01           1.0000000
S    1   1.00
      8.204444e-02           1.0000000
S    1   1.00
      2.244417e-02           1.0000000
S    1   1.00
      6.908733e-03           1.0000000
S    1   1.00
      2.252960e-03           1.0000000
S    1   1.00
      6.972017e-04           1.0000000
P    1   1.00
      5.9940000              1.0000000
P    1   1.00
      1.7450000              1.0000000
P    1   1.00
      0.5600000              1.0000000
P    1   1.00
      0.1626000              1.0000000
P    1   1.00
      0.0472                 1.0000000
P    1   1.00
      4.442081e-01           1.0000000
P    1   1.00
      9.422795e-02           1.0000000
P    1   1.00
      2.719846e-02           1.0000000
P    1   1.00
      8.859505e-03           1.0000000
P    1   1.00
      3.080030e-03           1.0000000
P    1   1.00
      1.066130e-03           1.0000000
D    1   1.00
      4.2990000              1.0000000
D    1   1.00
      1.2230000              1.0000000
D    1   1.00
      0.3510000              1.0000000
D    1   1.00
      0.101                  1.0000000
D    1   1.00
      3.348153e-01           1.0000000
D    1   1.00
      8.135966e-02           1.0000000
D    1   1.00
      2.631020e-02           1.0000000
D    1   1.00
      9.484616e-03           1.0000000
D    1   1.00
      3.635094e-03           1.0000000
D    1   1.00
      1.407186e-03           1.0000000
F    1   1.00
      2.6800000              1.0000000
F    1   1.00
      0.6906000              1.0000000
F    1   1.00
      0.178                  1.0000000
F    1   1.00
      3.463980e-01           1.0000000
F    1   1.00
      8.331040e-02           1.0000000
F    1   1.00
      2.805333e-02           1.0000000
F    1   1.00
      1.056283e-02           1.0000000
F    1   1.00
      4.235223e-03           1.0000000
F    1   1.00
      1.732138e-03           1.0000000
****
""",
}

In [ ]:
# Lets intiate AOint() a subclass of psi4utils() 
aoint = pyci.utils.AOint('d-aug-cc-pVQZ+ARO6f', '../.xyz/He.xyz', psi4mem='8 Gb', numpymem=8, custom_basis=True, basis_dict=basis_dict)
# saves all the necessary integrals as .npz files
aoint.save_all_aoints()
aoint.save_ao_dpints()
aoint.save_ao_qdints()
# run scf and get info for canonical HF orbitals
aoint.save_mo_info()
# lets load  ao_erints, mo_info and try to get the CIS matrix
ao_erints = np.load('.scratch/ao_erints.npz')['electron_repulsion_aoints'] 
eps_a = np.load('.scratch/mo_scf_info.npz')['eps_a']
Ca = np.load('.scratch/mo_scf_info.npz')['Ca'] 
ao_dipoles = np.load('.scratch/ao_dpints.npz')
ao_quadrupoles = np.load('.scratch/ao_qdints.npz')
# lets convert our erints from AO basis to MO basis
mo_erints = aoint.eri_ao2mo(Ca, ao_erints, greedy=True)
mo_dpx = aoint.matrix_ao2mo(Ca, ao_dipoles['dpx_aoints'])
mo_dpy = aoint.matrix_ao2mo(Ca, ao_dipoles['dpy_aoints'])
mo_dpz = aoint.matrix_ao2mo(Ca, ao_dipoles['dpz_aoints'])
# mo_qdxx = aoint.matrix_ao2mo(Ca, ao_quadrupoles['qdxx_aoints'])
# mo_qdyy = aoint.matrix_ao2mo(Ca, ao_quadrupoles['qdyy_aoints'])
# mo_qdzz = aoint.matrix_ao2mo(Ca, ao_quadrupoles['qdzz_aoints'])
del ao_erints, ao_dipoles, ao_quadrupoles

In [ ]:
nbf, nmo, nso, na, nb, nocc, nvirt = aoint.get_orb_info(aoint.wfn)

In [ ]:
scf_energy = aoint.scf_energy
psi4_cisd = psi4.energy('CISD')

In [ ]:
orbinfo = (nocc, nmo)
active_space = (nocc,nvirt)
options = { 'singles' : True,
            'full_cis' : True,
            'doubles' : True,
            'doubles_iiaa' : True,
            'doubles_iiab' : True,
            'doubles_ijaa' : True,
            'doubles_ijab_A' : True,
            'doubles_ijab_B' : True}
csfs, num_csfs = pyci.configint.rcisd.generate_csfs(orbinfo, active_space, options)
num_csfs, sum(num_csfs)

In [ ]:
HCISD = pyci.configint.rcisd.comp_hcisd(eps_a, mo_erints, scf_energy, orbinfo, active_space, options, ncore=4)

In [ ]:
HCISD0 = HCISD - scf_energy*np.eye(sum(num_csfs))
vals, vecs = np.linalg.eigh(HCISD0)
vals_scaled = vals + scf_energy

In [ ]:
vals_scaled[0], psi4_cisd, abs(vals_scaled[0] - psi4_cisd)

In [ ]:
csf_dpx  = pyci.configint.rcisd.comp_oeprop(mo_dpx, orbinfo, active_space, options)
csf_dpy  = pyci.configint.rcisd.comp_oeprop(mo_dpy, orbinfo, active_space, options)
csf_dpz  = pyci.configint.rcisd.comp_oeprop(mo_dpz, orbinfo, active_space, options)

In [ ]:
from pyci.integrators import RK4
from pyci.tdci import excite

In [ ]:
fs_to_au = 41.341374575751
ti = 0.0 
tf = 20.0*fs_to_au
dt = 0.0001*fs_to_au 
sigma = 5.1*fs_to_au
E0 = 0.07549
w0 = 0.056
phase = 0.0
params = E0, w0, ti, 2*sigma, sigma, phase
func = lambda t :  -1j * (HCISD0 - excite.sin2_pulse(t, params)*csf_dpz)
psi0 = vecs[0]
time_params =  ti, tf, dt
propagator = RK4(func, psi0, time_params)

In [ ]:
propagator._time_propagation(ops_list=[csf_dpx, csf_dpy, csf_dpz, HCISD], ops_headers=['dpx', 'dpy', 'dpz', 'energy'], print_nstep= 100, outfile='rk4_tdprop.txt')

In [ ]:
from pyci.integrators.splitoperator import SplitOperator 
fs_to_au = 41.341374575751
ti = 0.0 
tf = 20.0*fs_to_au
dt = 0.0001*fs_to_au 
sigma = 5.1*fs_to_au
E0 = 0.07549
w0 = 0.056
phase = 0.0
params = E0, w0, ti, 2*sigma, sigma, phase
func = lambda t : (excite.sin2_pulse(t, params)*csf_dpz)
psi0 = vecs[0]
time_params =  ti, tf, dt
propagator = SplitOperator(vals, vecs, func, psi0, time_params)

In [ ]:
propagator._time_propagation(ops_list=[csf_dpx, csf_dpy, csf_dpz, HCISD], ops_headers=['dpx', 'dpy', 'dpz', 'energy'], print_nstep= 100, outfile='so_tdprop.txt')

In [ ]:
from pyci.tdci.lifetime import heuristic
model = heuristic(vals_scaled, vecs, csfs, eps_a)
IP = 0.90357185430291 # 24.587387 eV
ip_params = E0, w0, IP
cmplx_vals = model.cmplx_energies(ip_params, use_d2=False)


In [ ]:
from pyci.integrators.splitoperator import SplitOperator 
fs_to_au = 41.341374575751
ti = 0.0 
tf = 20.0*fs_to_au
dt = 0.0001*fs_to_au 
sigma = 5.1*fs_to_au
E0 = 0.07549
w0 = 0.056
phase = 0.0
params = E0, w0, ti, 2*sigma, sigma, phase
func = lambda t : excite.sin2_pulse(t, params)*csf_dpz
psi0 = vecs[0]
time_params =  ti, tf, dt
propagator = SplitOperator(cmplx_vals, vecs, func, psi0, time_params)

In [ ]:
propagator._time_propagation(ops_list=[csf_dpx, csf_dpy, csf_dpz, HCISD], ops_headers=['dpx', 'dpy', 'dpz', 'energy'], print_nstep= 2000, outfile='lt_so_tdprop.txt')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
rk4_data = np.loadtxt('rk4_tdprop.txt',skiprows=1)
so_data = np.loadtxt('so_tdprop.txt',skiprows=1)
lt_so_data = np.loadtxt('lt_so_tdprop.txt',skiprows=1)
plt.figure(figsize=(10,8), facecolor='white')
plt.plot(rk4_data[:,0],rk4_data[:,4], '--', label='RK4 w/o lifetime')
plt.plot(so_data[:,0],so_data[:,4], '--', label='SO w/o lifetime')
plt.plot(lt_so_data[:,0],lt_so_data[:,4], '-.',label='w/ lifetime')
plt.legend()

In [ ]:
i = 5
plt.figure(figsize=(10,8), facecolor='white')
plt.plot(rk4_data[:,0], rk4_data[:,i]-rk4_data[0,i], '--', label='RK4 w/o lifetime')
plt.plot(so_data[:,0], so_data[:,i]-so_data[0,i], '--', label='SO w/o lifetime')
plt.plot(lt_so_data[:,0], lt_so_data[:,i]-lt_so_data[0,i], '-.',label='w/ lifetime')
plt.legend()

In [ ]:
from numpy import fft
from pyci.utils.units import fs_to_au

def get_prop_data(filename):
    prop_data = np.loadtxt(filename, skiprows=1)
    time_fs = prop_data[:, 0]
    norm = prop_data[:, 1]
    X, Y, Z = prop_data[:, 2], prop_data[:, 3], prop_data[:, 4]
    energy = prop_data[:, 5]
    # Fx, Fy, Fz = prop_data[:, 7], prop_data[:, 8], prop_data[:, 9]
    dipoles = (X,Y,Z)
    # fields = (Fx, Fy, Fz)
    return time_fs, dipoles, energy, norm

def calc_moments(time_fs, observable):
    time = time_fs * fs_to_au
    velocity = np.gradient(observable, time)
    acceleration = np.gradient(velocity, time)
    return velocity, acceleration

def calc_fft(time_fs, observable, dt_fs=1e-4, return_moments=False,
             velocity=[0], acceleration=[0]):
    dt_fs = time_fs[2]-time_fs[1]
    dt = dt_fs * fs_to_au
    time = time_fs * fs_to_au
    freq = 2*np.pi*fft.fftfreq(len(time), dt) 
    obs_fft = fft.fft(observable)
    if return_moments:
        obs_vel_fft = fft.fft(velocity)
        obs_acc_fft = fft.fft(acceleration)
        return freq, obs_fft, obs_vel_fft, obs_acc_fft
    else:
        return freq, obs_fft

def calc_Gobs(time_fs, observable, 
              return_moments=False, obs_vel=[0], obs_acc=[0], dt_fs=1e-4):
    freq, obs_fft, obs_vel_fft, obs_acc_fft = calc_fft(time_fs, observable, dt_fs=dt_fs,
                                                       return_moments=True,
                                                       velocity=obs_vel,
                                                       acceleration=obs_acc)
    G_obs = abs(1/((time_fs[-1]-time_fs[0])*fs_to_au) * obs_fft)**2 
    G_obs_vel = abs(1/((time_fs[-1]-time_fs[0])*fs_to_au) * obs_vel_fft)**2 
    G_obs_acc = abs(1/((time_fs[-1]-time_fs[0])*fs_to_au) * obs_acc_fft)**2 
    if return_moments:
        return freq, G_obs, G_obs_vel, G_obs_acc
    else:
        return freq, G_obs

def plot_hhg(freq, G_obs, G_obs_vel, G_obs_acc, axes, label, xmax=15, w0=0.056961578478002):
    n = int(freq.shape[0]/2)
    hhg =  (G_obs)[:n]
    hhg_vel =  (G_obs_vel)[:n]
    hhg_acc =  (G_obs_acc)[:n]

    index_w0 = max(range(len(hhg)), key=hhg.__getitem__)
    index_w0_vel = max(range(len(hhg_vel)), key=hhg_vel.__getitem__)
    index_w0_acc = max(range(len(hhg_acc)), key=hhg_vel.__getitem__)

    ho = freq[:n]/w0 
    ho_vel = freq[:n]/w0 
    ho_acc = freq[:n]/w0 

    axes[0].plot(ho, hhg, linewidth=1.0, color='red', label=label+r': $\frac{1}{|t_{f}- t_{i}|}\int \langle {D_{z}}(t)\rangle e^{i\omega t} dt$')
    axes[0].legend(frameon=False, loc='upper right')
    axes[0].set_yscale('log')
    axes[0].set_xlim(0,xmax)
    axes[0].tick_params(labelbottom=False)
    axes[1].plot(ho_vel, hhg_vel, linewidth=1.0, color='blue', label=label+r': $\frac{1}{|t_{f}- t_{i}|}\int \langle \dot{D_{z}}(t)\rangle e^{i\omega t} dt$')
    axes[1].legend(frameon=False, loc='upper right')
    axes[1].set_yscale('log')
    axes[1].set_xlim(0,xmax)
    axes[1].tick_params(labelbottom=False)
    axes[1].set_ylabel(r'Signal in (a.u.)', size=18)
    axes[2].plot(ho_acc, hhg_acc, linewidth=1.0, color='orange', label=label+r': $\frac{1}{|t_{f}- t_{i}|}\int \langle \ddot{D_{z}}(t)\rangle e^{i\omega t} dt$')
    axes[2].legend(frameon=False, loc='upper right')
    axes[2].set_yscale('log')
    axes[2].set_xlim(0,xmax)
    axes[2].set_xlabel(r'harmonic order ($\omega / \omega_{0}$)', size=18)



In [ ]:
def get_hhg_data(filename):
    time_fs, dipoles, energy, norm = get_prop_data(filename)
    dpz = dipoles[2]
    dpz_vel, dpz_acc = calc_moments(time_fs,dpz)
    freq, G_obs, G_obs_vel, G_obs_acc = calc_Gobs(time_fs, dpz, 
                                                return_moments=True, 
                                                obs_vel=dpz_vel, 
                                                obs_acc=dpz_acc, dt_fs=1e-4)
    return freq, G_obs, G_obs_vel, G_obs_acc     

fig, axes = plt.subplots(3,2, facecolor='white',figsize=(12,10))
freq_so, G_obs_so, G_obs_vel_so, G_obs_acc_so = get_hhg_data('so_tdprop.txt')
freq_lt_so, G_obs_lt_so, G_obs_vel_lt_so, G_obs_acc_lt_so = get_hhg_data('lt_so_tdprop.txt')
plot_hhg(freq_so, G_obs_so, G_obs_vel_so, G_obs_acc_so, axes[:,0], xmax=25, label='w/o lifetimes')
plot_hhg(freq_lt_so, G_obs_lt_so, G_obs_vel_lt_so, G_obs_acc_lt_so, axes[:,1],xmax=25, label='w/ lifetimes')

In [ ]:
Ecut = IP + 3.17*(E0/(4*w0))**2
print(Ecut, Ecut/w0)


In [ ]:
plt.plot(vals_scaled)